In [5]:
from dotenv import load_dotenv
import os

In [6]:
load_dotenv("../.env")

True

In [7]:
api_key = os.getenv("ALPHA_VANTAGE_API_KEY")

## Intraday Stock Market Data API

In [13]:
function = "TIME_SERIES_INTRADAY"
symbol = "MSFT"
interval = "5min"
datatype = "csv"
outputsize = "full"

In [10]:
from datetime import datetime

current_year = datetime.now().year

In [16]:
import requests

for i in [current_year - 2, current_year - 1, current_year]:
    for j in range(1, 13):
        period = f"{i}-{j:02d}"
        endpoint = f"https://www.alphavantage.co/query?function={function}&symbol={symbol}&interval={interval}&outputsize={outputsize}&apikey={api_key}&datatype={datatype}&month={period}"
        r = requests.get(url=endpoint)

        if not os.path.exists(f"../data/{i}/{j:02d}"):
            os.makedirs(f"../data/{i}/{j:02d}")
            
        with open(f"../data/{i}/{j:02d}/{symbol}_{interval}.csv", "w") as f:
            f.write(r.text)


## News & Sentiments API

In [18]:
function = "NEWS_SENTIMENT"
tickers = "MSFT,Microsoft"
time_from = "20230101T0400"
time_to = "20250115T1955"

endpoint = f"https://www.alphavantage.co/query?function={function}&tickers={tickers}&apikey={api_key}&time_from={time_from}&time_to={time_to}"

r = requests.get(url=endpoint)


In [19]:
r.json()

{'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}